In [120]:
# Crawling에 필요한 module 선언
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# hover 해야 나오는 데이터 크롤링을 위한 모듈 추가(201116)
from selenium.webdriver.common.action_chains import ActionChains

# (201105) 추가한 모듈
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# lxml
from lxml import html
import xml.etree.ElementTree as ET

# pandas
import pandas as pd

# requests, json, re
import requests
import json
import re

In [121]:
# 메인에서 유튜브 a tag url에서 utbID를 긁어오는 함수 정의
def main_video_id_scrap_list():
    # webdriver define
    driver_path = './chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(30) # or bigger second
    
    # 열고자 하는 url 정의
    url = "https://www.youtube.com/"
    
    # 해당 url로 창 열기
    driver.get(url)
    time.sleep(5)
    
    # 유튜브 video ID값을 저장할 리스트 정의
    youtube_id_list = []
    
    # 첫 동영상 시작 시 시작번호(index = 1)
    index = 1
    SCROLL_PAUSE_TIME = 2
    
    while True:
        try:
            youtube_video_id_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[' + str(index) + ']/div/ytd-rich-grid-media/div[1]/div/div[1]/h3/a'
            
            # 해당 페이지의 8개의 영상을 모두 확인했다면 그다음으로 가기위해 스크롤
            if index % 8 == 0:
                # 2초 대기 (대기 시간은 크롤러의 정밀도에 따라 조정 가능)
                driver.execute_script("window.scrollBy(0, 1080);")
                time.sleep(SCROLL_PAUSE_TIME)
                driver.execute_script("window.scrollBy(0, 1080);")
                time.sleep(SCROLL_PAUSE_TIME)
                driver.execute_script("window.scrollBy(0, 1080);")
                time.sleep(SCROLL_PAUSE_TIME)
            
            youtube_title_link = driver.find_element_by_xpath(youtube_video_id_xpath)
            youtube_id = youtube_title_link.get_attribute('href')
            youtube_id = youtube_id.split("v=")[1].split("&")[0]
            youtube_id_list.append(youtube_id)
            
            # 리스트에 들어간 데이터 중간 확인
            print("index_no: ", index)
            print("youtube_id: ", youtube_id)
            
            index += 1
            
        except Exception as e:
            print()
            print("Exception: ", e)
            break
    # youtube_id 수집이 끝난 브라우저 닫기        
    driver.close()
    
    return youtube_id_list

In [122]:
youtube_id_list = main_video_id_scrap_list()
print("Making youtube_id_list: ", youtube_id_list)


Exception:  Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/h3/a"}
  (Session info: chrome=86.0.4240.198)

Making youtube_id_list:  []


In [132]:
# 유튜브 동영상 id list
youtube_id_list = []

# 유튜브 정식 카테고리 k, v dictionary
youtube_category_dict = {
    'Film & Animation' : 1,
    'Autos & Vehicles' : 2,
    'Music' : 10,
    'Pets & Animals' : 15,
    'Sports' : 17,
    'Short Movies' : 18,
    'Travel & Events' : 19,
    'Gaming' : 20,
    'Videoblogging' : 21,
    'People & Blogs' : 22,
    'Comedy' : 23,
    'Entertainment' : 24,
    'News & Politics' : 25,
    'Howto & Style' : 26,
    'Education' : 27,
    'Science & Technology' : 28,
    'Movies' : 30,
    'Anime/Animation' : 31,
    'Action/Adventure' : 32,
    'Classics' : 33,
    'Documentary' : 35,
    'Drama' : 36,
    'Family' : 37,
    'Foreign' : 38,
    'Horror' : 39,
    'Sci-Fi/Fantasy' : 40,
    'Thriller' : 41,
    'Shorts' : 42,
    'Shows' : 43,
    'Trailers' :44
}

In [184]:
def scrap_detail_video_data(video_id_list):
    
    # webdriver 초기화
    driver_path = './chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(30) # or bigger second
    
    # origin_url 정의
    origin_url = 'https://www.youtube.com/watch?v='
    
    # 테스트용 단일 youtube_id 정의
    # (임시) 딘일로 넣어주는 값
    video_id = 'y57Zldv2bik'
    
    # youtube_detail_raw_data를 받을 dictionary 정의
    youtube_raw_data = {}
    
    # 넣어주는 url 값
    url = origin_url + video_id
    
    # scriptTag에서 긁어지는 데이터
    script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'
    
    # meta tag에서 긁어지는 데이터 -> soup.find, soup.findAll 이용해서 긁기
    request = requests.get(url)
    soup = BeautifulSoup(request.text, 'lxml')
    
    # ytInitialData에서 긁어지는 데이터 -> 복잡한 계층의 json 데이터 기반
    yt_initial_data_xpath = '/html/body/script[19]'
    
    # 수집할 데이터의 리스트
    title_list = []
    author_list = []
    description_list = []
    url_list = []
    thumb_url_list = []
    like_cnt_list = []
    dislike_cnt_list = []
    follow_cnt_list = []
    view_cnt_list = []
    duration_list = []
    youtube_category_list = []
    youtube_category_sn_list = []
    is_family_friendly_list = []
    title_hash_tags_list = []
    description_hash_tags_list = []
    video_id_list = [] 
    author_id_list = []
    yupload_time_list = []
    keywords_list = []
    
    driver.get(url)
    time.sleep(5)
    
    # Data 크롤링
    # 각 유형별로 다른 로직으로 데이터를 크롤링한다
    
    # script tag id=scriptTag의 내부 HTML 내용을 가져온다(scriptTag data crawling)
    script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

    script_tag_json = json.loads(script_tag_str)
    
    time.sleep(5)

    # print("script_tag_json: ", script_tag_json)
    # print("script_tag_json_type: ", type(script_tag_json))

    # Data result test
    title = script_tag_json['name']
    author = script_tag_json['author']
    description = script_tag_json['description']
    thumb_url = script_tag_json['thumbnailUrl'][0]
    view_cnt = int(script_tag_json['interactionCount'])
    duration = int(script_tag_json['duration'][2:len(script_tag_json['duration'])-1].replace(",", ""))
    youtube_category = script_tag_json['genre']
    # addition : youtube_category_sn
    youtube_category_sn = youtube_category_dict[genre]

    print("title: ", title)
    print("author: ", author)
    print("description: ", description)
    print("thumb_url: ", thumb_url)
    print("view_cnt: ", view_cnt, " type: ", type(view_cnt))
    print("duration: ", duration, " type: ", type(duration))
    print("youtube_category: ", youtube_category)
    print("youtube_category_id: ", youtube_category_sn)
    
    # scriptTag data appending
    title_list.append(title)
    author_list.append(author)
    description_list.append(description)
    url_list.append(url)
    thumb_url_list.append(thumb_url)
    view_cnt_list.append(view_cnt)
    duration_list.append(duration)
    youtube_category_list.append(youtube_category)
    youtube_category_sn_list.append(youtube_category_sn)
    
    
    # meta tag data crawling
    is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
    author_id = soup.find("meta", {"itemprop" : "channelId"})['content']
    yupload_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
    keywords = ",".join([tag['content'] for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

    print("is_family_friendly: ", is_family_friendly)
    print("video_id: ", video_id)
    print("author_id: ", author_id)
    print("yupload_time: ", yupload_time)
    print("keywords: ", keywords)
    
    # meta tag data appending
    is_family_friendly_list.append(is_family_friendly)
    video_id_list.append(video_id)
    author_id_list.append(author_id)
    yupload_time_list.append(yupload_time)
    keywords_list.append(keywords)
    
    # ytInitialData crawling
    yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
    yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('window["ytInitialData"] =', '').strip()
    # print("yt_initial_data: ", yt_initial_data_str)

    time.sleep(5)

    # data to json
    # json loading이 간헐적으로 안되는 현상 수정 필요
    yt_initial_data_json = json.loads(yt_initial_data_str)

    # print("yt_initial_data_json: ", yt_initial_data_json)

    # video_info_renderer
    video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
    video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
    video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

    # print("video_primary_info_renderer: ", video_primary_info_renderer)
    # print("video_secondary_info_renderer: ", video_secondary_info_renderer)

    # 좋아요 & 싫어요
    like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

    print("like_dislike_cnt: ", like_dislike_cnt)
    
    like_cnt = int(like_dislike_cnt.split(" / ")[0].replace(",", ""))
    dislike_cnt = int(like_dislike_cnt.split(" / ")[1].replace(",", ""))
    
    print("like_cnt: ", like_cnt, " type: ", type(like_cnt))
    print("dislike_cnt: ", dislike_cnt, " type: ", type(dislike_cnt))

    # 제목 타이틀 해시태그
    title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
    title_hash_tag_list = []
    for tag in title_hash_tags:
        if tag['text'] not in [" ", ""]:
            title_hash_tag_list.append(tag['text'][1:])
            # print("title_hash_tag: ", tag['text'][1:])

    title_hash_tags_str = ",".join(title_hash_tag_list)
    print("title_hash_tags_str: ", title_hash_tags_str)

    # 내용 해시태그
    description_hash_tags = video_secondary_info_renderer['description']['runs']
    description_hash_tag_list = []
    for tag in description_hash_tags:
        if tag['text'] not in [" ", ""]:
            if "#" in tag['text']:
                description_hash_tag_list.append(tag['text'][1:])
                # print("description_hash_tag: ", tag['text'][1:])

    description_hash_tags_str = ",".join(description_hash_tag_list)
    print("description_hash_tags_str: ", description_hash_tags_str)

    # 구독자 수
    raw_follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text'][4:]
    follow_cnt = raw_follow_cnt[:len(raw_follow_cnt)-1]
    
    # 구독자 수 수치화 필터링
    num_count_dict = {
        '천' : 1000,
        '만' : 10000,
        '억' : 100000000,
        '조' : 1000000000000
    }
    
    # print("follow_cnt: ", follow_cnt)
    # print("follow_num_cnt: ", follow_cnt[-1])
    
    follow_cnt_num = int(follow_cnt[:len(follow_cnt)-1]) * num_count_dict[follow_cnt[-1]]
    
    print("follow_cnt_num: ", follow_cnt_num, " type: ", type(follow_cnt_num))
    
    # yInitialData data appending
    like_cnt_list.append(like_cnt)
    dislike_cnt_list.append(dislike_cnt)
    title_hash_tags_list.append(title_hash_tags_str)
    description_hash_tags_list.append(description_hash_tags_str)
    follow_cnt_list.append(follow_cnt_num)
    
    driver.close()
    
    # 얻어온 데이터를 dictionary에 저장
    youtube_raw_data['title'] = title_list
    youtube_raw_data['authorNick'] = author_list
    youtube_raw_data['description'] = description_list
    youtube_raw_data['urlPath'] = url_list
    youtube_raw_data['thumbUrl'] = thumb_url_list
    youtube_raw_data['likeCnt'] = like_cnt_list
    youtube_raw_data['dislikeCnt'] = dislike_cnt_list
    youtube_raw_data['followCnt'] = follow_cnt_list
    youtube_raw_data['viewCnt'] = view_cnt_list
    youtube_raw_data['duration'] = duration_list
    youtube_raw_data['utbCategory'] = youtube_category_list
    youtube_raw_data['utbCateSN'] = youtube_category_sn_list
    youtube_raw_data['isFamilyFriendly'] = is_family_friendly_list
    youtube_raw_data['titleHashTags'] = title_hash_tags_list
    youtube_raw_data['descHashTags'] = description_hash_tags_list
    youtube_raw_data['utbVideoID'] = video_id_list
    youtube_raw_data['authorID'] = author_id_list
    youtube_raw_data['yUploadDateTime'] = yupload_time_list
    youtube_raw_data['keywords'] = keywords_list
    
    # DataFrame 생성
    df = pd.DataFrame(youtube_raw_data)
    
    return df
    # 모아둔 DataFrame을 csv로 변환

In [186]:
# 크롤러 실행
scrap_detail_video_data(youtube_id_list)

title:  [다시보기] 검찰, 월성1호기 조기폐쇄 수사 속도 | 2020년 11월 11일 뉴스A 라이브
author:  채널A 뉴스
description:  [뉴스A 라이브] 채널A 뉴스A 라이브 (12:00~13:20)

검찰, 감사원 ‘백데이터’ 확보…7000쪽 분석

2018년 4월 초 청와대·산업부에서 무슨 일이?

법원, 윤석열 부인 의혹 압수 영장 모두 기각

‘신규·갱신’ 2배까지 벌어진 서울 아파트 전세가

“전두환 호헌조치 우려”…1987년 바이든의 편지

바이든의 미소…“미국이 돌아왔다”

[세상터치]이번엔 ‘은주’…한국계 세번째 / ‘흙수저’ 부부의 쾌거

‘16개월 영아 학대 사망’ 엄마 구속 심사 外

[날씨]맑고 일교차 커…동해안 등 연일 건조특보

#A라이브 #뉴스 #뉴스alive

▷ 공식 홈페이지 http://www.ichannela.com
▷ 공식 페이스북 https://www.facebook.com/channelanews
▷ 공식 유튜브 https://www.youtube.com/tvchanews
thumb_url:  https://i.ytimg.com/vi/y57Zldv2bik/maxresdefault.jpg
view_cnt:  190639  type:  <class 'int'>
duration:  3382  type:  <class 'int'>
youtube_category:  News & Politics
youtube_category_id:  25
is_family_friendly:  True
video_id:  y57Zldv2bik
author_id:  UCfq4V1DAuaojnr2ryvWNysw
yupload_time:  2020-11-10
keywords:  채널A,채널A 뉴스,채널A NEWS,뉴스A
like_dislike_cnt:  3,719 / 222
like_cnt:  3719  type:  <class 'int'>
dislike_cnt:  222  type:  <class 'int'>
title_hash_

,title,authorNick,description,urlPath,thumbUrl,likeCnt,dislikeCnt,followCnt,viewCnt,duration,utbCategory,utbCateSN,isFamilyFriendly,titleHashTags,descHashTags,utbVideoID,authorID,yUploadDateTime,keywords
0,"[다시보기] 검찰, 월성1호기 조기폐쇄 수사 속도 | 2020년 11월 11일 뉴스...",채널A 뉴스,"[뉴스A 라이브] 채널A 뉴스A 라이브 (12:00~13:20)\n\n검찰, 감사원...",https://www.youtube.com/watch?v=y57Zldv2bik,https://i.ytimg.com/vi/y57Zldv2bik/maxresdefau...,3719,222,1020000,190639,3382,News & Politics,25,True,"A라이브,뉴스,뉴스alive","A라이브,뉴스,뉴스alive",y57Zldv2bik,UCfq4V1DAuaojnr2ryvWNysw,2020-11-10,"채널A,채널A 뉴스,채널A NEWS,뉴스A"


In [75]:
# scriptTag test

# By using BeautifulSoup
# url = "https://www.youtube.com/watch?v=XqZsoesa55w"
# request = requests.get(url)

# tree = html.fromstring(request.content)

# # scriptTag
# script_tag = tree.xpath('//script[@id="scriptTag"]')

# print("script_tag: ", script_tag)

# soup = BeautifulSoup(request.text, 'lxml')

# # script_tag = soup.find_all("script")
# script_tag = soup.select("script#scriptTag")

# print(len(script_tag))
# print("script_tag: ", script_tag)


# webdriver 초기화
driver_path = './chromedriver'
driver = webdriver.Chrome(driver_path)
driver.implicitly_wait(30) # or bigger second

# origin_url 정의
origin_url = 'https://www.youtube.com/watch?v='

# 테스트용 단일 youtube_id 정의
youtube_id = 'y57Zldv2bik'

script_tag_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[1]/ytd-player-microformat-renderer/script'

url = origin_url + youtube_id
print("making url: ", url)

driver.get(url)
time.sleep(5)

# Data scrap

# script tag id=scriptTag의 내부 HTML 내용을 가져온다
script_tag_str = driver.find_element_by_xpath(script_tag_xpath).get_attribute("innerHTML")

# print("script_tag_str: ", script_tag_str)
# print("script_tag_str_type: ", type(script_tag_str))

script_tag_json = json.loads(script_tag_str)

print("script_tag_json: ", script_tag_json)
print("script_tag_json_type: ", type(script_tag_json))

# Data result test
title = script_tag_json['name']
author = script_tag_json['author']
description = script_tag_json['description']
thumb_url = script_tag_json['thumbnailUrl'][0]
interaction_cnt = script_tag_json['interactionCount']
duration = script_tag_json['duration']
genre = script_tag_json['genre']

print("title: ", title, " author: ", author)
print("description: ", description)
print("thumb_url: ", thumb_url, " interaction_cnt: ", interaction_cnt, " duration: ", duration)
print("genre: ", genre)

time.sleep(5)
driver.close()

making url:  https://www.youtube.com/watch?v=y57Zldv2bik
script_tag_json:  {'@context': 'https://schema.org', '@type': 'VideoObject', 'description': '[뉴스A 라이브] 채널A 뉴스A 라이브 (12:00~13:20)\n\n검찰, 감사원 ‘백데이터’ 확보…7000쪽 분석\n\n2018년 4월 초 청와대·산업부에서 무슨 일이?\n\n법원, 윤석열 부인 의혹 압수 영장 모두 기각\n\n‘신규·갱신’ 2배까지 벌어진 서울 아파트 전세가\n\n“전두환 호헌조치 우려”…1987년 바이든의 편지\n\n바이든의 미소…“미국이 돌아왔다”\n\n[세상터치]이번엔 ‘은주’…한국계 세번째 / ‘흙수저’ 부부의 쾌거\n\n‘16개월 영아 학대 사망’ 엄마 구속 심사 外\n\n[날씨]맑고 일교차 커…동해안 등 연일 건조특보\n\n#A라이브 #뉴스 #뉴스alive\n\n▷ 공식 홈페이지 http://www.ichannela.com\n▷ 공식 페이스북 https://www.facebook.com/channelanews\n▷ 공식 유튜브 https://www.youtube.com/tvchanews', 'duration': 'PT3382S', 'embedUrl': 'https://www.youtube.com/embed/y57Zldv2bik', 'interactionCount': '165669', 'name': '[다시보기] 검찰, 월성1호기 조기폐쇄 수사 속도 | 2020년 11월 11일 뉴스A 라이브', 'thumbnailUrl': ['https://i.ytimg.com/vi/y57Zldv2bik/maxresdefault.jpg'], 'uploadDate': '2020-11-10', 'genre': 'News & Politics', 'author': '채널A 뉴스', 'publication': [{'@type': 'BroadcastEvent', 'isLiveBroadcast'

In [64]:
# meta tag test
url = "https://www.youtube.com/watch?v=y57Zldv2bik"
request = requests.get(url)
soup = BeautifulSoup(request.text, 'lxml')

# print("find_all_video_tag: ", soup.find_all("meta", {"property" : "og:video:tag"}))

is_family_friendly = soup.find("meta", {"itemprop" : "isFamilyFriendly"})['content']
channel_id = soup.find("meta", {"itemprop" : "channelId"})['content']
yupload_date_time = soup.find("meta", {"itemprop" : "uploadDate"})['content']
tag_string = ",".join([tag['content'] for tag in soup.find_all("meta", {"property" : "og:video:tag"})])

print("is_family_friendly: ", is_family_friendly)
print("channel_id: ", channel_id)
print("yupload_date_time: ", yupload_date_time)

print("tag_string: ", tag_string)

is_family_friendly:  True
channel_id:  UCfq4V1DAuaojnr2ryvWNysw
yupload_date_time:  2020-11-10
tag_string:  채널A,채널A 뉴스,채널A NEWS,뉴스A


In [110]:
# ytInitialData scrap test

# webdriver 초기화
driver_path = './chromedriver'
driver = webdriver.Chrome(driver_path)
driver.implicitly_wait(30) # or bigger second

# origin_url 정의
origin_url = 'https://www.youtube.com/watch?v='

# 테스트용 단일 youtube_id 정의
youtube_id = 'y57Zldv2bik'

# ytInitialData xpath
yt_initial_data_xpath = '/html/body/script[19]'

url = origin_url + youtube_id
print("making url: ", url)

driver.get(url)
time.sleep(5)

# Data scrap

yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('window["ytInitialData"] =', '').strip()
# print("yt_initial_data: ", yt_initial_data_str)

time.sleep(5)

# data to json
# json loading이 간헐적으로 안되는 현상 수정 필요
yt_initial_data_json = json.loads(yt_initial_data_str)

# print("yt_initial_data_json: ", yt_initial_data_json)


# video_info_renderer
video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

# print("video_primary_info_renderer: ", video_primary_info_renderer)
# print("video_secondary_info_renderer: ", video_secondary_info_renderer)

# 좋아요 & 싫어요
like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

print("like_dislike_cnt: ", like_dislike_cnt)

# 제목 타이틀 해시태그
title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
title_hash_tag_list = []
for tag in title_hash_tags:
    if tag['text'] not in [" ", ""]:
        title_hash_tag_list.append(tag['text'][1:])
        print("title_hash_tag: ", tag['text'][1:])

title_hash_tags_str = ",".join(title_hash_tag_list)
print("title_hash_tags_str: ", title_hash_tags_str)

# 내용 해시태그
description_hash_tags = video_secondary_info_renderer['description']['runs']
description_hash_tag_list = []
for tag in description_hash_tags:
    if tag['text'] not in [" ", ""]:
        if "#" in tag['text']:
            description_hash_tag_list.append(tag['text'][1:])
            print("description_hash_tag: ", tag['text'][1:])

description_hash_tags_str = ",".join(description_hash_tag_list)
print("description_hash_tags_str: ", description_hash_tags_str)

# 구독자 수
follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text']
print("follow_cnt: ", follow_cnt)
print("follow_num_cnt: ", follow_cnt[-1])

time.sleep(5)
driver.close()

making url:  https://www.youtube.com/watch?v=y57Zldv2bik
like_dislike_cnt:  3,630 / 217
title_hash_tag:  A라이브
title_hash_tag:  뉴스
title_hash_tag:  뉴스alive
title_hash_tags_str:  A라이브,뉴스,뉴스alive
description_hash_tag:  A라이브
description_hash_tag:  뉴스
description_hash_tag:  뉴스alive
description_hash_tags_str:  A라이브,뉴스,뉴스alive
follow_cnt:  구독자 102만명


In [111]:
# ytInitialData scrap test

# webdriver 초기화
driver_path = './chromedriver'
driver = webdriver.Chrome(driver_path)
driver.implicitly_wait(30) # or bigger second

# origin_url 정의
origin_url = 'https://www.youtube.com/watch?v='

# 테스트용 단일 youtube_id 정의
youtube_id = 'VcIs0Lvbsj4'

# ytInitialData
yt_initial_data_xpath = '/html/body/script[19]'

url = origin_url + youtube_id
print("making url: ", url)

driver.get(url)
time.sleep(5)

# Data scrap

# 좋아요와 싫어요는 ' / '를 기준으로 분리가 되어 있으며 그 값을 각각 split으로 나눠 받는다
# like_dislike_cnt = driver.find_element_by_xpath(like_dislike_cnt_xpath)
# hov = ActionChains(driver).move_to_element(like_dislike_cnt)
# hov.perform()
# like_dislike = like_dislike_cnt.text
# print("like_dislike: ", like_dislike_cnt.text)
# follow 수는 정확한 수치는 나오지 않는다. 명 앞에 붙은 단위를 검정하여 0을 덧붙이는 식으로 수치를 구성한다
yt_initial_data = driver.find_element_by_xpath(yt_initial_data_xpath)
# yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split('window["ytInitialData"] = ')[1]
yt_initial_data_str = yt_initial_data.get_attribute("innerHTML").split(";")[0].replace('window["ytInitialData"] =', '').strip()
# print("yt_initial_data: ", yt_initial_data_str)

time.sleep(5)

# data to json
# json loading이 간헐적으로 안되는 현상 수정 필요
yt_initial_data_json = json.loads(yt_initial_data_str)

# print("yt_initial_data_json: ", yt_initial_data_json)


# video_info_renderer
video_info_renderer = yt_initial_data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents']
video_primary_info_renderer = video_info_renderer[0]['videoPrimaryInfoRenderer']
video_secondary_info_renderer = video_info_renderer[1]['videoSecondaryInfoRenderer']

# print("video_primary_info_renderer: ", video_primary_info_renderer)
# print("video_secondary_info_renderer: ", video_secondary_info_renderer)

# 좋아요 & 싫어요
like_dislike_cnt = video_primary_info_renderer['sentimentBar']['sentimentBarRenderer']['tooltip']

print("like_dislike_cnt: ", like_dislike_cnt)

# 제목 타이틀 해시태그
title_hash_tags = video_primary_info_renderer['superTitleLink']['runs']
title_hash_tag_list = []
for tag in title_hash_tags:
    if tag['text'] not in [" ", ""]:
        title_hash_tag_list.append(tag['text'][1:])
        print("title_hash_tag: ", tag['text'][1:])

title_hash_tags_str = ",".join(title_hash_tag_list)
print("title_hash_tags_str: ", title_hash_tags_str)

# 내용 해시태그
description_hash_tags = video_secondary_info_renderer['description']['runs']
description_hash_tag_list = []
for tag in description_hash_tags:
    if tag['text'] not in [" ", ""]:
        if "#" in tag['text']:
            description_hash_tag_list.append(tag['text'][1:])
            print("description_hash_tag: ", tag['text'][1:])

description_hash_tags_str = ",".join(description_hash_tag_list)
print("description_hash_tags_str: ", description_hash_tags_str)

# 구독자 수
follow_cnt = video_secondary_info_renderer['owner']['videoOwnerRenderer']['subscriberCountText']['runs'][0]['text']

time.sleep(5)
driver.close()

making url:  https://www.youtube.com/watch?v=VcIs0Lvbsj4
like_dislike_cnt:  39,330 / 1,525
title_hash_tag:  커피와어울리는음악
title_hash_tag:  카페에서듣는재즈
title_hash_tag:  듣기좋은재즈
title_hash_tags_str:  커피와어울리는음악,카페에서듣는재즈,듣기좋은재즈
description_hash_tag:  커피와어울리는음악
description_hash_tag:  카페에서듣는재즈
description_hash_tag:  듣기좋은재즈
description_hash_tag:  듣기좋은음악
description_hash_tag:  재즈추천
description_hash_tag:  재즈음악
description_hash_tag:  카페재즈
description_hash_tag:  재즈음악모음
description_hash_tag:  커피와어울리는재즈
description_hash_tag:  카페음악
description_hash_tag:  카페에서듣기좋은음악
description_hash_tag:  힐링음악
description_hash_tag:  RelaxingJazz
description_hash_tag:  JazzMusic
description_hash_tag:  CafeJazz
description_hash_tags_str:  커피와어울리는음악,카페에서듣는재즈,듣기좋은재즈,듣기좋은음악,재즈추천,재즈음악,카페재즈,재즈음악모음,커피와어울리는재즈,카페음악,카페에서듣기좋은음악,힐링음악,RelaxingJazz,JazzMusic,CafeJazz
follow_cnt:  구독자 14.9만명


In [180]:
len("Science & Technology")

20